In [1]:
#import useful packages

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

import seaborn as sns
sns.set()

mpl.rcParams['lines.linewidth']=2.0
mpl.rcParams['xtick.labelsize']=13
mpl.rcParams['ytick.labelsize']=13
mpl.rcParams['axes.labelsize']=15
mpl.rcParams['axes.labelweight']='heavy'
mpl.rcParams['axes.titlesize']=18
mpl.rcParams['axes.titleweight']='heavy'
mpl.rcParams['legend.fontsize']=12



from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score

from sklearn.pipeline import Pipeline

import category_encoders as ce

import os
import sys

In [2]:
src_dir = os.path.join(os.path.dirname(os.getcwd()))
print(src_dir)
sys.path.append(src_dir)

from adfraud import models

/Users/peter/Documents/data_projects/adfraud


In [3]:
data_dir = os.path.join(os.path.dirname(os.getcwd()),'data')

data_filename = 'train_sample.csv'
data_location = os.path.join(data_dir,data_filename)
ad_data = pd.read_csv(data_location)
#feats_df  = ad_data[['ip','app','channel']]
#labels = ad_data['is_attributed'].values

In [4]:

ad_data['click_time'] = pd.to_datetime(ad_data['click_time'],format='%Y-%m-%d %H:%M:%S')
ad_data['click_time'] = ad_data['click_time'].dt.hour 

feats_df = ad_data[['ip','app','channel','click_time','os','device']]
labels   = ad_data['is_attributed'].values


In [5]:
train_test_splitter = StratifiedShuffleSplit(n_splits=1,test_size=0.1,random_state=42)
split = train_test_splitter.split(np.zeros((len(labels),)),labels)
train_inds,test_inds = next(split)
x_train = feats_df.iloc[train_inds]
y_train = labels[train_inds]
x_test  = feats_df.iloc[test_inds]
y_test  = labels[test_inds] 

In [7]:
param_grid = {
    'forest__n_estimators':[50],
    'forest__max_depth': [10],
    'forest__class_weight': [
                           {0:0.001,1:0.999}],
    'hash__n_components': [50]
}

In [10]:
model1 = models.HashForest(['ip','app','channel'],['app','channel'])
model1.train_CV(x_train,y_train,param_grid,n_splits=3)
auc1 = model1.test_auc(x_test,y_test)

model2 = models.HashForest(['ip','app','channel','click_time'],['app','channel','click_time'])
model2.train_CV(x_train,y_train,param_grid,n_splits=3)
auc2 = model2.test_auc(x_test,y_test)


Fitting 3 folds for each of 1 candidates, totalling 3 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV] forest__class_weight={0: 0.001, 1: 0.999}, forest__max_depth=10, forest__n_estimators=50, hash__n_components=50 
[CV]  forest__class_weight={0: 0.001, 1: 0.999}, forest__max_depth=10, forest__n_estimators=50, hash__n_components=50, score=0.921, total=  54.4s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   54.4s remaining:    0.0s
[CV] forest__class_weight={0: 0.001, 1: 0.999}, forest__max_depth=10, forest__n_estimators=50, hash__n_components=50 
[CV]  forest__class_weight={0: 0.001, 1: 0.999}, forest__max_depth=10, forest__n_estimators=50, hash__n_components=50, score=0.970, total=  54.2s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  1.8min remaining:    0.0s
[CV] forest__class_weight={0: 0.001, 1: 0.999}, forest__max_depth=10, forest__n_estimators=50, hash__n_components=50 
[CV]  forest__class_weight={0: 0.001, 1: 0.999}, fores

In [12]:
auc2 = model2.test_auc(x_test,y_test)
print(auc1,auc2)
print(model1.train_auc)

0.9614744346780202 0.9283874650827337
0.9289746246758085
